In [5]:
from tkinter import *
import cv2
import threading
from datetime import datetime
import requests
import os

cap = cv2.VideoCapture(0)

if not os.path.exists("image_log.csv"):
    with open("image_log.csv", "w") as f:
        f.write("ImageName,Timestamp,Latitude,Longitude\n")


def get_location_from_ip():
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if "loc" in data:
            lat, lon = data["loc"].split(",")
            return float(lat), float(lon)
        else:
            return None, None
    except Exception as e:
        print("Error retrieving location:", e)
        return None, None

def capture_image():
    img_counter = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        cv2.imshow('Frame', frame)
        key = cv2.waitKey(1)

        if key == ord('q'):
            break
        elif key % 256 == 32:  
            frame_with_metadata = frame.copy()

          
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S (%A)")
            cv2.putText(frame_with_metadata, timestamp, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (0,0,255), 2, cv2.LINE_AA)
            
            latitude, longitude = get_location_from_ip()
            latlon_text = f"Lat: {latitude}, Lon: {longitude}"
            cv2.putText(frame_with_metadata, latlon_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (0,0,255), 2, cv2.LINE_AA)
            
        
            img_name = f"Monika_{img_counter}.jpg"
            cv2.imwrite(img_name, frame_with_metadata)
            print(f"Image Captured: {img_name}")
            
         
            with open("image_log.csv", "a") as f:
                f.write(f"{img_name},{timestamp},{latitude},{longitude}\n")
            
            img_counter += 1

    cap.release()
    cv2.destroyAllWindows()


def start_capture():
    threading.Thread(target=capture_image).start()


window = Tk()
window.geometry("300x300")
window.title("GPS Camera")
label = Label(window, text="Geo Tagged Image Capture", font=("Bold", 18), fg="Black")
label.pack() 
button = Button(window, text="Capture Image", command=start_capture)
button.pack()

window.mainloop()


Image Captured: Monika_0.jpg
Image Captured: Monika_1.jpg
